In [ ]:
import sys
sys.path.append('../qick/qick_lib/')

from mkids import *

import numpy as np

import matplotlib.pyplot as plt
from numpy.fft import fft, fftshift

In [ ]:
# Initialize Firmware.
soc = MkidsSoc('./mkids_4x4_v1.bit')

# Name the Chains.
soc['analysis'][0]['name']  = 'ADC224 CH0, Low Frequency Balun'
soc['analysis'][1]['name']  = 'ADC224 CH1, Low Frequency Balun'
soc['analysis'][2]['name']  = 'ADC224 CH2, High Frequency Balun'
soc['analysis'][3]['name']  = 'ADC224 CH3, High Frequency Balun'
soc['synthesis'][0]['name'] = 'DAC229 CH0, High Frequency Balun'
soc['synthesis'][1]['name'] = 'DAC229 CH1, High Frequency Balun'
soc['synthesis'][2]['name'] = 'DAC229 CH2, Low Frequency Balun'
soc['synthesis'][3]['name'] = 'DAC229 CH3, Low Frequency Balun'

# Print information.
print(soc)

In [ ]:
################################
### Processing Chain Example ###
################################
# Build processing chain.
#chain = KidsChain(soc, soc['analysis'][1], soc['synthesis'][3], name = "Cable Loop-Back")
chain = KidsChain(soc, soc['analysis'][0], soc['synthesis'][2], name = "LPF 630")

print("Built MKIDs Processing Chain")
print(" ")

print("Analysis Chain: {}".format(chain.analysis.name))
print("  * sampling frequency = {} MHz".format(chain.analysis.fs))
print("  * channel separation = {} MHz".format(chain.analysis.fc_ch))
print(" ")

print("Syntheis Chain: {}".format(chain.synthesis.name))
print("  * sampling frequency = {} MHz".format(chain.synthesis.fs))
print("  * channel separation = {} MHz".format(chain.synthesis.fc_ch))
print(" ")

In [ ]:
#######################
### Frequency Sweep ###
#######################
f,a,phi=chain.sweep(100,104,N=100)

plt.figure(dpi=150)
plt.plot(f,20*np.log10(a/max(a)))
plt.ylim([-60,10])
plt.xlabel("Frequency [MHz]");
plt.ylabel("Amplitude [dB]");

plt.figure(dpi=150)
plt.plot(f,phi)
plt.xlabel("Frequency [MHz]");
plt.ylabel("Phase [rad]");

In [ ]:
############################
### Wide Frequency Sweep ###
############################
fstart = 0
fend = 2000

# Number of pointes per sweep.
N = 100

# Use 80 % the available bandwidth per sweep.
fbw = 0.8*min(chain.analysis.fs,chain.synthesis.fs)

if (fend-fstart)>fbw:
    fstart = np.arange(fstart, fend, fbw)

# Total number  of points.
NT = len(fstart)*N

f_v = np.zeros(NT)
a_v = np.zeros(NT)
phi_v = np.zeros(NT)
for i,ff in enumerate(fstart):
    fend_ = ff+fbw
    if fend_ > fend:
        fend_ = fend
    print("i = {}, fstart = {} MHz, fend = {} MHz.".format(i, ff, fend_))
    
    # Sweep.
    f,a,phi=chain.sweep(ff,fend_,N=N)
    
    # Concat values.
    f_v[i*N:(i+1)*N] = f
    a_v[i*N:(i+1)*N] = a
    phi_v[i*N:(i+1)*N] = phi
    
plt.figure(dpi=150)
plt.plot(f_v,20*np.log10(a_v/max(a_v)))
plt.xlabel("Frequency [MHz]");
plt.ylabel("Amplitude [dB]");

#plt.figure(dpi=150)
#plt.plot(f_v,phi_v)
#plt.xlabel("Frequency [MHz]");
#plt.ylabel("Phase [rad]");

In [ ]:
########################
### Delay estimation ###
########################
plot = True

# Set mixer.
chain.set_mixer_frequency(100)

# Starting frequency points.
fstart = np.linspace(100,500,5)

# Delta frequency per sweep.
df = 0.2

# Number of points per sweep.
N = 200

df_v = np.zeros(len(fstart))
dt_v = np.zeros(len(fstart))

for i in range(len(fstart)):
    dict_v[i] = {}
for i, ff in enumerate(fstart):
    # Frequency Sweep.
    f,a,phi=chain.sweep(ff,ff+df,N=N, set_mixer=False)
    df_v[i], dt_v[i] = chain.phase_slope(f, phi)
    
    dict_v[i]['freq'] = f
    dict_v[i]['phi']  = phi
    
    print(" ")
    print("i = {}, fstart = {} MHz, df = {} MHz, dt = {} us".format(i, ff, df_v[i], dt_v[i]))
    
    if plot:
        #plt.figure(dpi=150)
        #plt.plot(f,20*np.log10(a/max(a)))
        #plt.ylim([-60,10])
        #plt.xlabel("Frequency [MHz]");
        #plt.ylabel("Amplitude [dB]");
        #plt.title("Iteration i = {}".format(i))

        plt.figure(dpi=150)
        plt.plot(f,phi)
        plt.xlabel("Frequency [MHz]");
        plt.ylabel("Phase [rad]");
        plt.title("Iteration i = {}".format(i))

In [ ]:
plt.figure(dpi=150);
for i in range(len(dict_v)):
    plt.plot(dict_v[i]['freq'] - dict_v[i]['freq'][0], dict_v[i]['phi'], label="i = {}".format(i))
    
plt.legend()

In [ ]:
dict_v[1]

In [ ]:
################################
### Multiple Synthesis Chain ###
################################
synt = [SynthesisChain(soc, soc['synthesis'][0]), SynthesisChain(soc, soc['synthesis'][3])]

# Sampling frequency.
fs=synt[0].dict['chain']['fs']

fmix = 500
for s in synt:
    # Set mixer frequency.
    s.set_mixer_frequency(fmix)

    # All channels off.
    s.alloff()

    # Set output quantization.
    s.qout(3)

    fv = np.linspace(fmix,fmix+10,7)
    for f in fv:
        print("f = {} MHz".format(f))
        s.set_tone(f, g=0.02)

In [ ]:
#########################
### Spectrum Analyzer ###
#########################

fmix = 100
chain.set_mixer_frequency(fmix)

# Set some output tones.
fv = np.linspace(fmix,fmix+10,7)
#for f in fv:
#    print("f = {} MHz".format(f))
    #chain.synthesis.set_tone(f, g=0.2)
    
# Data source.
chain.analysis.source("input")

# Decimation.
chain.analysis.set_decimation(1)

# Frequency range.
fstart = 130
fend   = 136
f = np.arange(fstart, fend, chain.analysis.fc_ch)
#f = [110,111,112]
print("Spectrum")
print("fstart = {} MHz, fend = {} MHz, fc = {} MHz".format(fstart, fend, chain.analysis.fc_ch))

# Set mixer to starting point.
chain.analysis.set_mixer_frequency(-fstart)

# Frequency and amplitude vectors.
FF = []
AA = []
plt.figure(dpi=150);
for i,fck in enumerate(f):
    print("i = {}, fck = {} MHz".format(i,fck))
    
    # Transfer data.
    [xi,xq] = chain.analysis.get_bin(fck)    
    x = xi + 1j*xq
    
    # Frequency vector.
    F = (np.arange(len(x))/len(x)-0.5)*chain.analysis.fs_ch
    
    # Normalization factor.
    NF = (2**15)*len(F)

    w = np.hanning(len(x))
    xw = x*w
    YY = fftshift(fft(xw))
    YYlog = 20*np.log10(abs(YY)/NF)
    AA = np.concatenate((AA,YYlog))
    
    Fk = F+fck
    FF = np.concatenate((FF,Fk))
    plt.plot(Fk,YYlog);
#plt.plot(FF,AA);

In [ ]:
###############################
### Zooming-in the spectrum ###
###############################
# Set decimation.
chain.analysis.set_decimation(100)

# Set quantization.
chain.analysis.qout(2)

# Data source.
chain.analysis.source("product")

# Get data.
fc = 132.4
[xi,xq] = chain.analysis.get_bin(fc,verbose=True)
xi = xi[100:]
xq = xq[100:]
x = xi + 1j*xq

w = np.hanning(len(x))
xw = x*w
F = (np.arange(len(x))/len(x)-0.5)*chain.analysis.fs_ch*1000
Y = fftshift(fft(xw))

plt.figure(1,dpi=150)
plt.plot(F,20*np.log10(abs(Y)))
plt.xlabel("F [kHz]");

In [ ]:
f = 110.51
chain.synthesis.alloff()
chain.synthesis.set_tone(f, g=0.1)

In [ ]:
########################
### Delay estimation ###
########################
# Delta frequency per sweep.
df = 0.1

# Number of points per sweep.
N = 200

# Starting frequency points.
fstart = 100.5 - df/2
#fstart = 110.5 - 10*chain.fr

#df = N*chain.fr

# Frequency Sweep.
f,a,phi = chain.sweep(fstart,fstart+df,N=N, decimation=1,set_mixer=False, verbose=False)
df, dt = chain.phase_slope(f, phi)
 
print(" ")
print("df = {} MHz, dt = {} us".format(df, dt))

plt.figure(dpi=150)
plt.plot(f,phi)
plt.xlabel("Frequency [MHz]");
plt.ylabel("Phase [rad]");

In [ ]:
##############################
### Phase Correction by DT ###
##############################
DT = 20.2

plt.figure(dpi=150)
plt.plot(f,phi)
plt.title("Original Phase");


# Unwrap phase.
phi_u = np.unwrap(phi)
phi_u = phi_u - phi_u[0]

plt.figure(dpi=150)
plt.plot(f,phi_u)
plt.title("Unwrap Phase");

# Correct phase.
# phi_u = 2*pi*f*(DT + dt)
# phi_u = 2*pi*f*DT + 2*pi*f*dt = phi_DT + phi_dt
# phi_dt = phi_u - 2*pi*f*DT
phi_dt = phi_u - 2*np.pi*f*DT
phi_dt = phi_dt - phi_dt[0]
plt.figure(dpi=150)
plt.plot(f,phi_dt)
plt.title("Correctred phase by DT = {} us".format(DT));

#plt.figure(dpi=150)
#plt.plot(f,phi_u,label='Unwrap')
#plt.plot(f,phi_dt,label='Corrected')
#plt.legend();

In [ ]:
##################################
### Poly Fit to estimate Delay ###
##################################
m = np.zeros(2)

off = 10
mid = int(len(f)/2)

x = f[off:mid-off]
y = phi_dt[off:mid-off]

coef = np.polyfit(x,y,1)
poly1d_fn = np.poly1d(coef)

plt.figure(dpi=150);
plt.plot(x,y, '.', x, poly1d_fn(x), '--k', label="Lower");

m[0] = coef[0]

print("Slope = {}".format(coef[0]))

x = f[mid+off:-off]
y = phi_dt[mid+off:-off]

coef = np.polyfit(x,y,1)
poly1d_fn = np.poly1d(coef)

plt.figure(dpi=150);
plt.plot(x,y, '.', x, poly1d_fn(x), '--k', label="Upper");

m[1] = coef[0]

print("Slope = {}".format(coef[0]))

# Slope ratio.
md = max(abs(m)) - min(abs(m))
print("Slope Difference = {}".format(md))
print("Time Difference = {} ns".format(1000*md/(2*np.pi)))

In [ ]:
# Poly fit.
x = f[550:950]
y = phi_dt[550:950]

coef = np.polyfit(x,y,1)
poly1d_fn = np.poly1d(coef)

plt.figure(dpi=150);
plt.plot(x,y, '.', x, poly1d_fn(x), '--k');

print("Slope = {}".format(coef[0]))

In [ ]:
chain.set_mixer_frequency(100)

analysis_ch = 10
analysis_freq = chain.analysis.ch2freq(analysis_ch)

synthesis_ch = chain.synthesis.freq2ch(analysis_freq)
synthesis_freq = chain.synthesis.ch2freq(synthesis_ch)

print("Analysis Channel = {}, Analysis Frequency = {} MHz".format(analysis_ch, analysis_freq))
print("Synthesis Channel = {}, Synthesis Frequency = {} MHz".format(synthesis_ch, synthesis_freq))

In [ ]:
# Set mixer frequency.
chain.set_mixer_frequency(100)

# Set decimation.
chain.analysis.set_decimation(100)

# Base dds frequency.
fd = 0.49
#fdds = chain.analysis.fs_ch/20 - chain.fr

# Channels.
ch0 = 4*8
freq0 = chain.analysis.ch2freq(ch0)

ch1 = 4*8+1
freq1 = chain.analysis.ch2freq(ch1)

# Set output tone.
chain.synthesis.alloff()
chain.synthesis.set_tone(freq0+fd, g=0.1,verbose=True)
#chain.synthesis.set_tone(freq0+fd, g=0.1,verbose=True)

# Mask all channels.
chain.analysis.maskall()

# Set analysis tone (dds mode).
chain.analysis.source("input")

# get_bin will configure the DDS to the specified frequency.
[xi0,xq0] = chain.analysis.get_bin(f=freq0,verbose=True)
[xi1,xq1] = chain.analysis.get_bin(f=freq1,verbose=True)

# Channel Sel block.
chsel = getattr(soc,chain.analysis.dict['chain']['chsel'])

# Get data (at the same time).
data = chain.analysis.get_data_all()

# Check only 1 transaction is enabled.
if len(chsel.dict['tran']) > 1:
    print("ERROR: more than one transaction is enabled")
    
ntran = chsel.dict['tran'][0]
#xi0 = data['samples'][ntran][0,:]
#xq0 = data['samples'][ntran][1,:]
#xi1 = data['samples'][ntran][2,:]
#xq1 = data['samples'][ntran][3,:]

In [ ]:
plt.figure(dpi=150)
plt.plot(xi0,label="channel 0")
plt.plot(xi1,label="channel 1")
plt.legend()

In [ ]:
plt.figure(dpi=150)
plt.plot(xq0,label="channel 0")
plt.plot(xq1,label="channel 1")
plt.legend()

In [ ]:
plt.figure(dpi=150)
plt.plot(xi0,label="channel 0")
plt.plot(xi1,label="channel 1")
plt.xlim([130,1000])
plt.legend()


In [ ]:
plt.figure(dpi=150)
n=np.arange(len(xq0))
plt.plot(n,xq0,label="channel 0")
plt.plot(n,xq1,label="channel 1")
plt.xlim([130,180])
plt.legend()

In [ ]:
1/chain.analysis.fs_ch

In [ ]:
1/.09

In [ ]:
########################
### Delay estimation ###
########################

chain.set_mixer_frequency(100)

# Delta frequency per sweep.
df = 4*chain.fr

# Number of points per sweep.
N = 4

# Starting frequency points.
fstart = 120.5 - 2*chain.fr
fend = 120.5 + 2*chain.fr

#df = N*chain.fr

# Frequency Sweep.
f,a,phi = chain.sweep(fstart,fend,N=N, decimation=1,set_mixer=False,verbose=True)
df, dt = chain.phase_slope(f, phi)
 
print(" ")
print("df = {} MHz, dt = {} us".format(df, dt))

plt.figure(dpi=150)
plt.plot(f,phi)
plt.xlabel("Frequency [MHz]");
plt.ylabel("Phase [rad]");

In [ ]:
fstart = 120.5 - 2*chain.fr
fend = 120.5 + 2*chain.fr
f = np.linspace(fstart,fend,N)

In [ ]:
int(np.round(f[2]/chain.fr))*chain.fr

In [ ]:
plt.figure(dpi=150)
for i,fi in enumerate(f):
    # Set output tone.
    chain.synthesis.alloff()
    chain.synthesis.set_tone(fi, g=0.5,verbose=True)

    # Mask all channels.
    chain.analysis.maskall()

    # Set analysis tone (dds mode).
    chain.analysis.source("product")
    chain.analysis.set_decimation(200)

    # get_bin will configure the DDS to the specified frequency.
    [xi,xq] = chain.analysis.get_bin(f=fi)
    
    #plt.figure(dpi=150)
    plt.plot(xi[100:],label="i = {}".format(i))
    #plt.plot(xq[100:],label="Q")
    
plt.legend()

In [ ]:
streamer = getattr(soc,chain.analysis.dict['chain']['streamer'])

In [ ]:
streamer.set(50000)

In [ ]:
fs=soc.adcs['00']['fs']/4
print("fs = {} MHz, ts = {} ns".format(fs,1000/fs))